In [ ]:
!pip install -U  langchain==0.2.11 openai==1.37.0 ragas==0.1.11 arxiv==2.1.3 pymupdf==1.24.9 chromadb==0.5.5 wandb==0.17.5 tiktoken==0.7.0 pypdf==4.3.1 sentence_transformers==2.7.0

In [3]:
import os
from langchain.document_loaders import pdf, PyPDFLoader

def load_pdf_doucuments(pdf_folder_path: str) -> list:
    base_docs = []
    
    if not os.path.exists(pdf_folder_path):
        raise FileNotFoundError(f"The folder '{pdf_folder_path}' does not exist.")

    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            print(f"Processing: {pdf_path}")
            try:
                loader = PyPDFLoader(pdf_path)
                pages = loader.load()
                base_docs.extend(pages)
            except Exception as e:
                print(f"Error processing {pdf_path}: {str(e)}")

    return base_docs

In [4]:
# 设置pdf文件夹路径, 请将<1>替换为pdf文件夹路径, 文件夹名为RiceBlastData
pdf_folder_path = 'RiceBlastData'
base_docs = load_pdf_doucuments(pdf_folder_path)
len(base_docs)

Processing: RiceBlastData/20-100 (1).pdf


Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet


Processing: RiceBlastData/20-100.pdf


Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet


Processing: RiceBlastData/202004-411-422.pdf
Processing: RiceBlastData/2021-null.pdf
Processing: RiceBlastData/2022-null.pdf


...

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)


Processing: RiceBlastData/水稻稻瘟病-百度百科.pdf
Processing: RiceBlastData/稻瘟病-百度百科.pdf


Ignoring wrong pointing object 6 0 (offset 0)


Processing: RiceBlastData/稻瘟病菌.pdf
Processing: RiceBlastData/稻瘟病菌群体遗传结构的研究进展.pdf


97

In [ ]:
for doc in base_docs:
  print(doc.metadata)

In [9]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# bgm-m3路径位于$GEMINI_PRETRAIN下
EMBEDDING_PATH = os.path.expandvars('$GEMINI_PRETRAIN/bge-m3')
# 加载bge-m3模型
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_PATH)

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 补充创建一个RecursiveCharacterTextSplitter分割器，并且设定每个分块（chunk）的大小为1000，块间重叠部分的token数为100;
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 使用创建的分割器对base_docs里存的所有pdf文档进行分割，分割后的chunks保存在docs中；
docs = text_splitter.split_documents(base_docs)

# 将分割后的chunks，通过前面加载的embedding模型转换为向量，并存储在向量库vectorstore中。
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [12]:
len(docs)
import csv

with open("splited_docs.csv", 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 遍历数据写入
    for row in docs:
        writer.writerow(row)

In [13]:
print(max([len(chunk.page_content) for chunk in docs]))

1000


In [14]:
# 使用vectorstore.as_retriever方法创建检索器，注意设定参数为检索最相关的Top2文本块。
base_retriever = vectorstore.as_retriever(top_k=2)

relevant_docs = base_retriever.get_relevant_documents("什么是稻瘟病?")

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [15]:
len(relevant_docs)
print(relevant_docs)

[Document(metadata={'page': 0, 'source': 'RiceBlastData/稻瘟病-百度百科.pdf'}, page_content='稻瘟病  稻瘟病⼜名稻热病、⽕烧瘟、叩头瘟等，是由稻瘟病原菌引起的、发⽣在⽔稻的⼀种病害。稻瘟病在⽔稻整个⽣育期中都可发⽣，为害秧苗、叶⽚、穗、节等，分别称为苗瘟、叶瘟、穗瘟和节瘟。 稻瘟病分布遍及世界稻区，是稻作⽣产中的主要病害，其中以亚洲、⾮洲稻区发病为重。在中国地区⼀般⼭区重于平原，粳、糯重于籼稻，除华南稻区早稻重于晚稻外，其他稻区晚稻重于早稻。流⾏年份，⼀般减产10-20%，重的达40-50%，局部⽥块甚⾄颗粒⽆收。 稻瘟病的防治⽅法主要有选⽤抗病品种， 培育优质秧苗、 肥⽔管理技术措施、 加强⽥间管理、落实防治措施、化学药剂控制等。在稻瘟病流⾏以后，要切实掌握稻瘟病的发病症状、发病规律和发病原因， 结合种植情况， 采取科学可⾏的稻瘟病防治技术措施， 树⽴起良好的预防理念，降低稻瘟病发⽣的可能性，避免给稻⾕产量和质量造成损失。 2023年3⽉7⽇，被农业农村部列⼊《⼀类农作物病⾍害名录（2023年） 》 。  病原特征 稻瘟病原菌属于半知菌类丛梗孢科、梨形孢属 [1]。稻瘟病病原菌⽆性阶段（⾃然条件下）为灰梨孢菌（学名：Pyricularia oryzae Cav.）， 属 半 知 菌 类 梨 形 孢 属 ； 有 性 阶 段 （ 在 ⼈ ⼯ 培 养基上经⽤不同菌株交配后形成的有性世代） 为灰⾊⼤⻆间座壳菌 （学名：Magnaporthe grisea Barr. ）， 属 ⼦ 囊 菌 ⼴ ⼤ ⻆ 间 座 壳 属 [4]。  形态特征 菌丝：⽆⾊透明，丝状，有隔膜 [1]。 分⽣孢⼦梗：3-5根丛⽣成束，有时多达10根，从⽓孔或病部表⽪伸出，线状不分⽀，⼤⼩为112（-456）微⽶×3（-4）微⽶，有2-8隔膜，基部稍膨⼤，略带褐⾊，越⾄上部⾊越淡， 其顶端可产⽣分⽣孢⼦5-6个， 多的达9-20余个， 顶端屈曲处有分⽣孢⼦脱落的痕迹。 分⽣孢⼦：⽆⾊透明，鸭梨形或慈姑形。成熟孢⼦通常有2个隔膜，顶端尖，基部钝圆，有⼩突起，⼤⼩为17（-33）微⽶×6.5（-11）微⽶，萌发时两端细胞产⽣芽管，芽管的顶端，产⽣附者胞，呈球形或椭圆形，深褐⾊，壁厚，直径8-12微⽶，能紧紧贴附于寄主体上，产⽣

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """请基于以下提供的上下文信息，回答如下问题。如果你认为根据提供的信息无法回答该问题，请回答'我不知道':

### 上下文信息
{context}

### 问题
问题: {question}
"""

# 调用ChatPromptTemplate的合适方法生成提问模板。
prompt = ChatPromptTemplate.from_template(template)

In [21]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class ChatGLM3_LLM(LLM):
    # 基于本地 ChatGLM3 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: ChatGLM3 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        # 从本地加载一个预训练的分词器（tokenizer）
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        # 从本地加载一个预训练的生成式语言模型
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).half().cuda()
        # 将模型设置为评估模式
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        response, history = self.model.chat(self.tokenizer, prompt , history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM3-6B"

In [23]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.chains import LLMChain
from langchain.schema.messages import AIMessage
from langchain_community.llms.chatglm3 import ChatGLM3
from langchain_core.prompts import PromptTemplate

# 设定chatglm3-6b模型的地址
model_path = os.path.expandvars("$GEMINI_PRETRAIN2/chatglm3-6b")

# 初始化一个ChatGLM3_LLM的实例
primary_qa_llm = ChatGLM3_LLM(model_path)

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | base_retriever, 
        "question": itemgetter("question")
    } 
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {
        "response": prompt | primary_qa_llm | StrOutputParser(), 
        "context": itemgetter("context")
    }
)

正在从本地加载模型...


Loading checkpoint shards: 100%|██████████| 7/7 [02:11<00:00, 18.79s/it]


完成本地模型的加载


In [24]:
question = "抗稻瘟病的品种有哪些？"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': '抗稻瘟病的品种有宜恢 80（携带稻瘟病抗性基因 Pi9）、旗 3优 386（辽稻 20A 与携带 Pi-ta 基因的水稻配组选育而成）、川农优 538（携带 Pi-ta 基因的水稻与蜀恢 538 配组，选育出了抗病性强、稳产性好、米质优等特點的水稻新品种）以及宜恢 80（与携带 Pi-ta 基因的水稻连粳 05-45 杂交，培育出了携带 Pi-ta 和 Pi-2 基因组合的水稻）。', 'context': [Document(metadata={'page': 2, 'source': 'RiceBlastData/BP20230400000_87846733.pdf'}, page_content='治效果，发现将链霉菌 Ahn75和解淀粉芽孢杆菌 CWJ2菌株组合后对水稻叶瘟盆栽防效为 65.07%，对穗\n颈瘟的盆栽和田间防效分别为 63.00%、52.16% ，显著高于单一菌株[24] 。夏木凤等研究表明施用复合微\n生物菌剂有利于水稻生长发育，提高水稻产量[ 25]。 \n4.3. 种植抗病品种  \n随着生物技术的发展 ，种植抗稻瘟病品种是防治稻瘟病最根本、有效的方法。开发抗稻瘟病的水稻\n新品种、聚合两个或多个稻瘟病抗性基因，创制高产、高抗的优质水稻已成为水稻育种的重要目标[26] 。\n目前已发掘了 100余个稻瘟病抗性基因，已有 39个基因被克隆[27] 。以辐恢 838为受体亲本选育而成的\n恢复系宜恢 80，携带稻瘟病抗性基因 Pi9 [28]，是一个优良恢复系，可与不同来源的不育系杂交选育抗\n性品种。不育系旗 3A与恢复系福恢 386配组选育成的水稻新品种旗 3优386，中抗稻瘟病，同时具有株\n型适中、米质较优、抗倒伏等特点[29] 。高抗稻瘟病杂交水稻深两优 326由不育系深 08S与恢复系 R326\n配组选育而成[30] ；不育系川农 6A与恢复系蜀恢 538配组，选育出了抗病性强、稳产性好、米质优等特\n点的水稻新品种川农优 538 [31]，均为生产上提供了稳产的抗病新品种。 将携带 Pi-ta基因的水稻连粳\n05-45与携带 Pi-2基因的水稻 H401杂交配组，培育出了携带 Pi-ta和Pi-2基因组合的水稻，表现出较强\n的抗病水平[32] 。 \n参考文献  \n[1] Ainsworth

In [25]:
question = "什么是稻瘟病？"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': '稻瘟病是由稻瘟病原菌引起的、发生在水稻的一種病害。它會对水稻的秧苗、叶片、穗部等造成不同程度的影响，导致水稻产量降低。稻瘟病分布遍及世界稻区，是稻作產产中的主要病害，其中以亚洲、非洲稻区发病为重。在中国地区，一般山区重于平原，粳、糯重于籼稻，除华南稻区早稻重于晚稻外，其他稻区晚稻重于早稻。稻瘟病的防治方法主要有选用抗病品种，培育优质秧苗、肥水管理技术措施、加强田间管理、落实防治措施、化学药剂控制等。', 'context': [Document(metadata={'page': 0, 'source': 'RiceBlastData/稻瘟病-百度百科.pdf'}, page_content='稻瘟病  稻瘟病⼜名稻热病、⽕烧瘟、叩头瘟等，是由稻瘟病原菌引起的、发⽣在⽔稻的⼀种病害。稻瘟病在⽔稻整个⽣育期中都可发⽣，为害秧苗、叶⽚、穗、节等，分别称为苗瘟、叶瘟、穗瘟和节瘟。 稻瘟病分布遍及世界稻区，是稻作⽣产中的主要病害，其中以亚洲、⾮洲稻区发病为重。在中国地区⼀般⼭区重于平原，粳、糯重于籼稻，除华南稻区早稻重于晚稻外，其他稻区晚稻重于早稻。流⾏年份，⼀般减产10-20%，重的达40-50%，局部⽥块甚⾄颗粒⽆收。 稻瘟病的防治⽅法主要有选⽤抗病品种， 培育优质秧苗、 肥⽔管理技术措施、 加强⽥间管理、落实防治措施、化学药剂控制等。在稻瘟病流⾏以后，要切实掌握稻瘟病的发病症状、发病规律和发病原因， 结合种植情况， 采取科学可⾏的稻瘟病防治技术措施， 树⽴起良好的预防理念，降低稻瘟病发⽣的可能性，避免给稻⾕产量和质量造成损失。 2023年3⽉7⽇，被农业农村部列⼊《⼀类农作物病⾍害名录（2023年） 》 。  病原特征 稻瘟病原菌属于半知菌类丛梗孢科、梨形孢属 [1]。稻瘟病病原菌⽆性阶段（⾃然条件下）为灰梨孢菌（学名：Pyricularia oryzae Cav.）， 属 半 知 菌 类 梨 形 孢 属 ； 有 性 阶 段 （ 在 ⼈ ⼯ 培 养基上经⽤不同菌株交配后形成的有性世代） 为灰⾊⼤⻆间座壳菌 （学名：Magnaporthe grisea Barr. ）， 属 ⼦ 囊 菌 ⼴ ⼤ ⻆ 间 座 壳 属 [4]。  形态特征 菌丝：⽆⾊透明，丝状，有隔膜 [1]。 分⽣孢⼦梗：3-5根丛⽣成束，有时多达10根，从⽓孔或病部表⽪伸出，

In [29]:
question = "一个半小时是三个半小时吗？"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': '一个半小时不是三个半小时。一个半小时是指1小时30分钟，而三个半小时是指2小时15分钟。', 'context': [Document(metadata={'page': 3, 'source': 'RiceBlastData/2021-null.pdf'}, page_content='/G3E/G3F/G45/G21/G22/G27/G28/G2E/G2F/G30/G46/G47/G48/G49/G4A/G2D/G2C/G3A/G3B/G3C/G3D/G3E/G3F/G4B/G2E\n/G2F/G21/G22/G28/G29/G28/G23/G4C/G4D/G4E/G4F/G30/G37/G38 /G50/G2C/G3A/G3B/G3C/G3D/G26/G27/G32/G29/G2A\n/G2B/G2C/G51/G52/G2D/G2C/G3A/G3B/G3C/G3D/G3E/G3F/G36\n/G24/G53/G54/G55/G56/G54\n/G2C/G3A/G3B/G57/G58/G59/G5A/G5B/G5C/G2D/G5D/G2C/G3B/G59/G5E/G5F/G30/G60/G61/G45/G37\n/G62/G3C/G3B/G63/G64/G57/G65/G66/G5A/G67/G68/G2D/G69/G6A/G6B/G6C/G36/G6D/G43/G2C/G3A/G3B/G6E/G6F/G70/G71/G72/G73/G74/G30/G75/G76/G77/G3C/G3B/G63/G64/G2D/G3C/G3D/G78/G79/G7A/G7B/G7C/G7D/G7E/G2D/G7F/G32/G31/G80/G81/G82/G83/G84/G85/G86/G87\n/G2B/G28/G2A/G2C/G36/G3F/G44/G30/G88/G89/G4D/G8A/G8B\n/G8C/G22/G8D/G8E/G8F/G2D/G3C/G3B/G63/G64/G90/G91/G37/G62/G4B/G92/G93/G3C/G42/G7B/G7C/G43/G51/G26/G2D/G2C/G3A/G3B/G3C/G3D/G61/G64/G23/G36/G46/G47/G94/G6B/G95/G96/G3C/G2C/G3A/G3B/G